In [1]:
import numpy as np

In [2]:
# initialize weights and biases
# assume network has 2 input features, 3 neurons in the hidden layer, and 1 output neuron
# He initialization
def initialize_parameters(input_dim, hidden_dim, output_dim):
    np.random.seed(1)
    W1 = np.random.randn(hidden_dim, input_dim) * np.sqrt(2. / input_dim)
    b1 = np.zeros((hidden_dim, 1))
    W2 = np.random.randn(output_dim, hidden_dim) * np.sqrt(2. / hidden_dim)
    b2 = np.zeros((output_dim, 1))
    return W1, b1, W2, b2

In [3]:
# use sigmoid function as activation function and its derivative for backpropagation
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(A):
    return A * (1 - A)

In [4]:
# forward propagation step computes the output of network
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    return A2, A1

In [5]:
# use binary cross-entropy as our cost function
# to prevent overfitting, you can add L2 regularization to the cost function
def compute_cost(A2, Y, parameters, lambda_):
    m = Y.shape[1]
    W1, W2 = parameters
    cross_entropy_cost = -np.sum(Y * np.log(A2) + (1 - Y) * np.log(1 - A2)) / m
    L2_regularization_cost = (lambda_ / (2 * m)) * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    cost = cross_entropy_cost + L2_regularization_cost
    return cost

In [6]:
# backpropagation computes gradients and updates weights and biases
# to prevent overfitting, add L2 regularization to the gradients
def backpropagation(X, Y, A2, A1, W1, W2, lambda_):
    m = X.shape[1]
    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T) / m + (lambda_ / m) * W2
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m

    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * sigmoid_derivative(A1)
    dW1 = np.dot(dZ1, X.T) / m + (lambda_ / m) * W1
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m

    gradients = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return gradients

In [7]:
# use gradient descent to update the weights and biases
def update_parameters(parameters, gradients, learning_rate):
    W1, b1, W2, b2 = parameters
    dW1, db1, dW2, db2 = gradients["dW1"], gradients["db1"], gradients["dW2"], gradients["db2"]

    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    updated_parameters = (W1, b1, W2, b2)
    return updated_parameters

In [8]:
# normalize your inputs for better convergence
def normalize(X):
    mean = np.mean(X, axis=1, keepdims=True)
    std = np.std(X, axis=1, keepdims=True)
    return (X - mean) / std

In [9]:
# train network
# pass lambda for regularization and normalize data
def train(X, Y, input_dim, hidden_dim, output_dim, num_iterations, learning_rate, lambda_):
    X = normalize(X)
    W1, b1, W2, b2 = initialize_parameters(input_dim, hidden_dim, output_dim)

    for i in range(num_iterations):
        A2, A1 = forward_propagation(X, W1, b1, W2, b2)
        cost = compute_cost(A2, Y, (W1, W2), lambda_)
        gradients = backpropagation(X, Y, A2, A1, W1, W2, lambda_)
        parameters = (W1, b1, W2, b2)
        updated_parameters = update_parameters(parameters, gradients, learning_rate)
        W1, b1, W2, b2 = updated_parameters

        if i % 100 == 0:
            print(f"Iteration {i}: cost = {cost}")

    return W1, b1, W2, b2

In [10]:
# predictions
def predict(X, W1, b1, W2, b2):
    X = normalize(X)  # ensure input is normalized
    A2, _ = forward_propagation(X, W1, b1, W2, b2)
    return A2 > 0.5

In [11]:
# example dataset
X = np.array([[0, 0, 1, 1], [0, 1, 0, 1]])  # 2 features
Y = np.array([[0, 1, 1, 0]])  # Binary output

# Define hyperparameters
input_dim = 2
hidden_dim = 3
output_dim = 1
num_iterations = 1000
learning_rate = 0.01
lambda_ = 0.01  # Regularization parameter

# Train network
W1, b1, W2, b2 = train(X, Y, input_dim, hidden_dim, output_dim, num_iterations, learning_rate, lambda_)

# Predict
predictions = predict(X, W1, b1, W2, b2)
print("Predictions:", predictions)

Iteration 0: cost = 0.7780487377740158
Iteration 100: cost = 0.7501011519837623
Iteration 200: cost = 0.7362321743627279
Iteration 300: cost = 0.7285526943448185
Iteration 400: cost = 0.72358682203027
Iteration 500: cost = 0.7198580455171393
Iteration 600: cost = 0.7167426648540964
Iteration 700: cost = 0.7139720028984724
Iteration 800: cost = 0.7114243954086702
Iteration 900: cost = 0.7090395809550074
Predictions: [[False False  True  True]]


In [116]:
# The cost value is decreasing as training progresses, indicating that the model is learning and improving its performance.
# A decreasing cost suggests that the model's predictions are getting closer to the actual labels.